In [21]:
import pandas as pd
import csv
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [22]:
data = pd.read_csv("Root_Insurance_data.csv",delimiter = ",")
data_click_proba = pd.read_csv("data_click_probability.csv",delimiter = ",")

In [23]:
## Input: Data Frame (in the format of "Root_Insurance_data.csv") and encode_rank (default = False)
## Output: Data Frame (processed by one hot encoding ready to be used to make models)

## This function will one hot encode the following columns in the data frame: 
## Currently Insured, Marital Status, click, rank (optional, determined by encode_rank = True)
## and will drop the respective columns associated to the respective values of above columns:
## unknown, S, False, 5.0 (if encode_rank = True)

import category_encoders as ce

def prepare_data(data:pd.DataFrame(), encode_rank = False, click_proba = False) -> pd.DataFrame():
    temp = data.copy()
    
    if not click_proba:
        if encode_rank:
            one_hot_cols = ["Currently Insured", "Marital Status", "click", "rank"]
            drop_cols = ["Currently Insured_unknown", "Marital Status_S", "rank_5.0", "click_False"]
            column_names = ['Currently Insured_Y', 'Currently Insured_N', 'Number of Vehicles',
                            'Number of Drivers', 'Marital Status_M', 'bid', 'rank_1.0',
                            'rank_2.0', 'rank_3.0', 'rank_4.0', 'click_True', 'policies_sold']
        else:
            one_hot_cols = ["Currently Insured", "Marital Status", "click"]
            drop_cols = ["Currently Insured_unknown", "Marital Status_S", "click_False"]
            column_names = ['Currently Insured_Y', 'Currently Insured_N', 'Number of Vehicles','Number of Drivers',
                            'Marital Status_M', 'bid', 'rank', 'click_True', 'policies_sold']
    else:
        if encode_rank:
            one_hot_cols = ["Currently Insured", "Marital Status", "rank"]
            drop_cols = ["Currently Insured_unknown", "Marital Status_S", "rank_5.0"]
            column_names = ['Currently Insured_Y', 'Currently Insured_N', 'Number of Vehicles',
                            'Number of Drivers', 'Marital Status_M', 'bid', 'rank_1.0',
                            'rank_2.0', 'rank_3.0', 'rank_4.0', 'click', 'policies_sold']
        else:
            one_hot_cols = ["Currently Insured", "Marital Status"]
            drop_cols = ["Currently Insured_unknown", "Marital Status_S"]
            column_names = ['Currently Insured_Y', 'Currently Insured_N', 'Number of Vehicles','Number of Drivers',
                            'Marital Status_M', 'bid', 'rank', 'click', 'policies_sold']
    
    encoder = ce.OneHotEncoder(cols = one_hot_cols, use_cat_names = True)
    temp = encoder.fit_transform(temp)
    temp = temp.drop(columns = drop_cols)
    temp = temp.reindex(columns=column_names)
    return temp

In [24]:
#df = prepare_data(data, click_proba = False)


df = prepare_data(data_click_proba, click_proba = True)

C:\Users\David\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_curve, accuracy_score

features = list(df.columns)[:-1]
predictor_var = "policies_sold"

df_train, df_test = train_test_split(df,
                                     shuffle=True,
                                     random_state=12345,
                                     test_size=.2,
                                     stratify=df[predictor_var])

In [26]:
## Allows for variantion of resampling with condition resampling = "None", "Up", "Down" or "Balanced"

import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
import keras.backend as K
from sklearn.utils import resample,shuffle
from keras.utils.np_utils import to_categorical

def average_precision(y_true, y_pred):
    return K.sum(average_precision_score(y_true.numpy(), y_pred.numpy()))

def obtain_nn_scores(data:pd.DataFrame(), num_of_splits = 5, resampling = "None") -> []:
    
    avg_precision_cross_val = np.zeros(num_of_splits)
    roc_auc_cross_val = np.zeros(num_of_splits)
    accuracy_cross_val = np.zeros(num_of_splits)
    

    kfold = StratifiedKFold(n_splits=num_of_splits, shuffle=True, random_state=77)
    
    i = 0
    for train_index, test_index in kfold.split(df_train[features], df_train[predictor_var]):
        df_train_train = resample_data(df.iloc[train_index], resample_type = "None")
        df_train_val = df.iloc[test_index]

        model = models.Sequential()
        model.add(layers.Dense(len(features), activation='relu', input_dim= len(features)))
        model.add(layers.Dense(len(features), activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop', 
                      loss='binary_crossentropy',
                      metrics=[average_precision, 
                               keras.metrics.AUC(name = 'AUCROC'),
                               keras.metrics.BinaryAccuracy()],
                      run_eagerly=True)
        history = model.fit(df_train_train[features], 
                        df_train_train[predictor_var],
                        epochs = 200,
                        batch_size = 1024,
                        validation_data=(df_train_val[features], 
                                         df_train_val[predictor_var]))

        y_proba = model.predict(df_test[features])
        y_pred = [int(x > .5) for x in y_proba]
        
        avg_precision_cross_val[i] = average_precision_score(df_test[predictor_var], y_proba)
        roc_auc_cross_val[i] = roc_auc_score(df_test[predictor_var], y_proba)
        accuracy_cross_val[i] = accuracy_score(df_test[predictor_var], y_pred)
        i += 1
    
    return [avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val]

def resample_data(df:pd.DataFrame(), resample_type = "None") -> pd.DataFrame():
    df_minority = df[df[predictor_var] == 1]
    df_majority = df[df[predictor_var] != 1]
    if resample_type == "Up":
        df_minority_up = resample(df_minority,
                                  random_state=886,
                                  n_samples=len(df_majority),
                                  replace=True)
        df_resampled = pd.concat([df_minority_up, df_majority]).sample(frac=1, random_state = 0)
    elif resample_type == "Down":
        df_majority_down = resample(df_majority,
                                    random_state=886,
                                    n_samples=len(df_minority),
                                    replace=True)
        df_resampled = pd.concat([df_majority_down, df_minority]).sample(frac=1, random_state = 0)
    elif resample_type == "Balanced":
        df_majority_bal = resample(df_majority,
                                           random_state=886,
                                           n_samples= int(len(df)/2),
                                           replace=True)
        df_minority_bal = resample(df_minority,
                                               random_state=886,
                                               n_samples=int(len(df)/2),
                                               replace=True)
        df_resampled = pd.concat([df_majority_bal, df_minority_bal]).sample(frac=1, random_state = 0) 
    else:
        df_resampled = df
    return df_resampled

In [27]:
def info(avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val):
    print("Average cross val AUCPR = " + str(np.average(avg_precision_cross_val)))
    print()
    print("Average cross val AUCROC = " + str(np.average(roc_auc_cross_val)))
    print()
    print("Average cross val Accuracy = " + str(np.average(accuracy_cross_val)))

### No Resample Case

In [28]:
avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val = obtain_nn_scores(df_train)
print()
info(avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val)

Epoch 1/200
7/7 [==============================] - 0s 41ms/step - loss: 0.2463 - average_precision: 0.1834 - AUCROC: 0.7537 - binary_accuracy: 0.9211 - val_loss: 0.2209 - val_average_precision: 0.1263 - val_AUCROC: 0.7269 - val_binary_accuracy: 0.9344
Epoch 2/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2426 - average_precision: 0.1809 - AUCROC: 0.7674 - binary_accuracy: 0.9211 - val_loss: 0.2186 - val_average_precision: 0.1521 - val_AUCROC: 0.7485 - val_binary_accuracy: 0.9344
Epoch 3/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2410 - average_precision: 0.1993 - AUCROC: 0.7715 - binary_accuracy: 0.9211 - val_loss: 0.2180 - val_average_precision: 0.1528 - val_AUCROC: 0.7538 - val_binary_accuracy: 0.9344
Epoch 4/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2397 - average_precision: 0.1932 - AUCROC: 0.7780 - binary_accuracy: 0.9211 - val_loss: 0.2177 - val_average_precision: 0.1534 - val_AUCROC: 0.7528 - val_binary_accuracy:

7/7 [==============================] - 0s 27ms/step - loss: 0.2362 - average_precision: 0.2236 - AUCROC: 0.7847 - binary_accuracy: 0.9211 - val_loss: 0.2205 - val_average_precision: 0.1477 - val_AUCROC: 0.7527 - val_binary_accuracy: 0.9344
Epoch 34/200
7/7 [==============================] - 0s 30ms/step - loss: 0.2359 - average_precision: 0.2185 - AUCROC: 0.7863 - binary_accuracy: 0.9211 - val_loss: 0.2203 - val_average_precision: 0.1474 - val_AUCROC: 0.7517 - val_binary_accuracy: 0.9344
Epoch 35/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2361 - average_precision: 0.2152 - AUCROC: 0.7852 - binary_accuracy: 0.9211 - val_loss: 0.2193 - val_average_precision: 0.1471 - val_AUCROC: 0.7507 - val_binary_accuracy: 0.9344
Epoch 36/200
7/7 [==============================] - 0s 30ms/step - loss: 0.2359 - average_precision: 0.2316 - AUCROC: 0.7859 - binary_accuracy: 0.9211 - val_loss: 0.2211 - val_average_precision: 0.1478 - val_AUCROC: 0.7522 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 26ms/step - loss: 0.2356 - average_precision: 0.2141 - AUCROC: 0.7872 - binary_accuracy: 0.9211 - val_loss: 0.2199 - val_average_precision: 0.1481 - val_AUCROC: 0.7511 - val_binary_accuracy: 0.9344
Epoch 66/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2357 - average_precision: 0.2098 - AUCROC: 0.7870 - binary_accuracy: 0.9211 - val_loss: 0.2200 - val_average_precision: 0.1481 - val_AUCROC: 0.7531 - val_binary_accuracy: 0.9344
Epoch 67/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2354 - average_precision: 0.2064 - AUCROC: 0.7873 - binary_accuracy: 0.9211 - val_loss: 0.2199 - val_average_precision: 0.1482 - val_AUCROC: 0.7516 - val_binary_accuracy: 0.9344
Epoch 68/200
7/7 [==============================] - 0s 33ms/step - loss: 0.2355 - average_precision: 0.2238 - AUCROC: 0.7883 - binary_accuracy: 0.9211 - val_loss: 0.2199 - val_average_precision: 0.1482 - val_AUCROC: 0.7530 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 28ms/step - loss: 0.2354 - average_precision: 0.2111 - AUCROC: 0.7881 - binary_accuracy: 0.9211 - val_loss: 0.2195 - val_average_precision: 0.1476 - val_AUCROC: 0.7512 - val_binary_accuracy: 0.9344
Epoch 98/200
7/7 [==============================] - 0s 33ms/step - loss: 0.2354 - average_precision: 0.2111 - AUCROC: 0.7888 - binary_accuracy: 0.9211 - val_loss: 0.2191 - val_average_precision: 0.1476 - val_AUCROC: 0.7484 - val_binary_accuracy: 0.9344
Epoch 99/200
7/7 [==============================] - 0s 32ms/step - loss: 0.2354 - average_precision: 0.2249 - AUCROC: 0.7888 - binary_accuracy: 0.9211 - val_loss: 0.2208 - val_average_precision: 0.1477 - val_AUCROC: 0.7513 - val_binary_accuracy: 0.9344
Epoch 100/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2353 - average_precision: 0.2191 - AUCROC: 0.7880 - binary_accuracy: 0.9211 - val_loss: 0.2209 - val_average_precision: 0.1479 - val_AUCROC: 0.7509 - val_binary_accuracy: 0.9344


7/7 [==============================] - 0s 26ms/step - loss: 0.2353 - average_precision: 0.2336 - AUCROC: 0.7894 - binary_accuracy: 0.9211 - val_loss: 0.2191 - val_average_precision: 0.1479 - val_AUCROC: 0.7499 - val_binary_accuracy: 0.9344
Epoch 130/200
7/7 [==============================] - 0s 30ms/step - loss: 0.2353 - average_precision: 0.2176 - AUCROC: 0.7901 - binary_accuracy: 0.9211 - val_loss: 0.2210 - val_average_precision: 0.1480 - val_AUCROC: 0.7499 - val_binary_accuracy: 0.9344
Epoch 131/200
7/7 [==============================] - 0s 34ms/step - loss: 0.2353 - average_precision: 0.2324 - AUCROC: 0.7895 - binary_accuracy: 0.9211 - val_loss: 0.2222 - val_average_precision: 0.1476 - val_AUCROC: 0.7502 - val_binary_accuracy: 0.9344
Epoch 132/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2353 - average_precision: 0.2231 - AUCROC: 0.7893 - binary_accuracy: 0.9211 - val_loss: 0.2209 - val_average_precision: 0.1478 - val_AUCROC: 0.7500 - val_binary_accuracy: 0.934

...

7/7 [==============================] - 0s 26ms/step - loss: 0.2480 - average_precision: 0.1935 - AUCROC: 0.7702 - binary_accuracy: 0.9248 - val_loss: 0.2607 - val_average_precision: 0.2162 - val_AUCROC: 0.7712 - val_binary_accuracy: 0.9194
Epoch 66/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2476 - average_precision: 0.1999 - AUCROC: 0.7677 - binary_accuracy: 0.9248 - val_loss: 0.2602 - val_average_precision: 0.2165 - val_AUCROC: 0.7713 - val_binary_accuracy: 0.9194
Epoch 67/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2473 - average_precision: 0.2041 - AUCROC: 0.7695 - binary_accuracy: 0.9248 - val_loss: 0.2600 - val_average_precision: 0.2166 - val_AUCROC: 0.7722 - val_binary_accuracy: 0.9194
Epoch 68/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2470 - average_precision: 0.2222 - AUCROC: 0.7696 - binary_accuracy: 0.9248 - val_loss: 0.2596 - val_average_precision: 0.2166 - val_AUCROC: 0.7744 - val_binary_accuracy: 0.9194
E

7/7 [==============================] - 0s 24ms/step - loss: 0.2379 - average_precision: 0.1958 - AUCROC: 0.7760 - binary_accuracy: 0.9248 - val_loss: 0.2502 - val_average_precision: 0.2203 - val_AUCROC: 0.7730 - val_binary_accuracy: 0.9194
Epoch 98/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2376 - average_precision: 0.2016 - AUCROC: 0.7777 - binary_accuracy: 0.9248 - val_loss: 0.2501 - val_average_precision: 0.2203 - val_AUCROC: 0.7754 - val_binary_accuracy: 0.9194
Epoch 99/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2374 - average_precision: 0.2036 - AUCROC: 0.7764 - binary_accuracy: 0.9248 - val_loss: 0.2496 - val_average_precision: 0.2204 - val_AUCROC: 0.7761 - val_binary_accuracy: 0.9194
Epoch 100/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2371 - average_precision: 0.2098 - AUCROC: 0.7790 - binary_accuracy: 0.9248 - val_loss: 0.2493 - val_average_precision: 0.2210 - val_AUCROC: 0.7734 - val_binary_accuracy: 0.9194


7/7 [==============================] - 0s 27ms/step - loss: 0.2319 - average_precision: 0.2003 - AUCROC: 0.7810 - binary_accuracy: 0.9248 - val_loss: 0.2438 - val_average_precision: 0.2201 - val_AUCROC: 0.7773 - val_binary_accuracy: 0.9194
Epoch 130/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2317 - average_precision: 0.2033 - AUCROC: 0.7807 - binary_accuracy: 0.9248 - val_loss: 0.2439 - val_average_precision: 0.2201 - val_AUCROC: 0.7780 - val_binary_accuracy: 0.9194
Epoch 131/200
7/7 [==============================] - 0s 33ms/step - loss: 0.2317 - average_precision: 0.2068 - AUCROC: 0.7802 - binary_accuracy: 0.9248 - val_loss: 0.2436 - val_average_precision: 0.2201 - val_AUCROC: 0.7777 - val_binary_accuracy: 0.9194
Epoch 132/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2315 - average_precision: 0.2023 - AUCROC: 0.7803 - binary_accuracy: 0.9248 - val_loss: 0.2435 - val_average_precision: 0.2201 - val_AUCROC: 0.7780 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 25ms/step - loss: 0.2304 - average_precision: 0.1976 - AUCROC: 0.7807 - binary_accuracy: 0.9248 - val_loss: 0.2423 - val_average_precision: 0.2218 - val_AUCROC: 0.7785 - val_binary_accuracy: 0.9194
Epoch 162/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2304 - average_precision: 0.1968 - AUCROC: 0.7810 - binary_accuracy: 0.9248 - val_loss: 0.2423 - val_average_precision: 0.2203 - val_AUCROC: 0.7785 - val_binary_accuracy: 0.9194
Epoch 163/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2304 - average_precision: 0.2058 - AUCROC: 0.7804 - binary_accuracy: 0.9248 - val_loss: 0.2423 - val_average_precision: 0.2203 - val_AUCROC: 0.7778 - val_binary_accuracy: 0.9194
Epoch 164/200
7/7 [==============================] - 0s 38ms/step - loss: 0.2304 - average_precision: 0.1926 - AUCROC: 0.7810 - binary_accuracy: 0.9248 - val_loss: 0.2424 - val_average_precision: 0.2203 - val_AUCROC: 0.7783 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 40ms/step - loss: 0.2303 - average_precision: 0.2011 - AUCROC: 0.7809 - binary_accuracy: 0.9248 - val_loss: 0.2422 - val_average_precision: 0.2219 - val_AUCROC: 0.7787 - val_binary_accuracy: 0.9194
Epoch 194/200
7/7 [==============================] - 0s 43ms/step - loss: 0.2302 - average_precision: 0.2071 - AUCROC: 0.7806 - binary_accuracy: 0.9248 - val_loss: 0.2422 - val_average_precision: 0.2206 - val_AUCROC: 0.7782 - val_binary_accuracy: 0.9194
Epoch 195/200
7/7 [==============================] - 0s 46ms/step - loss: 0.2302 - average_precision: 0.1975 - AUCROC: 0.7816 - binary_accuracy: 0.9248 - val_loss: 0.2422 - val_average_precision: 0.2206 - val_AUCROC: 0.7782 - val_binary_accuracy: 0.9194
Epoch 196/200
7/7 [==============================] - 0s 47ms/step - loss: 0.2302 - average_precision: 0.1947 - AUCROC: 0.7805 - binary_accuracy: 0.9248 - val_loss: 0.2423 - val_average_precision: 0.2206 - val_AUCROC: 0.7776 - val_binary_accuracy: 0.919

### Upsample Case

In [29]:
avg_precision_cross_val_up, roc_auc_cross_val_up, accuracy_cross_val_up = obtain_nn_scores(df_train, resampling = "Up")
print()
info(avg_precision_cross_val_up, roc_auc_cross_val_up, accuracy_cross_val_up)

Epoch 1/200
7/7 [==============================] - 0s 55ms/step - loss: 3.1391 - average_precision: 0.1752 - AUCROC: 0.7368 - binary_accuracy: 0.0789 - val_loss: 2.9463 - val_average_precision: 0.1274 - val_AUCROC: 0.6986 - val_binary_accuracy: 0.0656
Epoch 2/200
7/7 [==============================] - 0s 40ms/step - loss: 2.7787 - average_precision: 0.1892 - AUCROC: 0.7493 - binary_accuracy: 0.0789 - val_loss: 2.6722 - val_average_precision: 0.1337 - val_AUCROC: 0.7042 - val_binary_accuracy: 0.0656
Epoch 3/200
7/7 [==============================] - 0s 31ms/step - loss: 2.5288 - average_precision: 0.1956 - AUCROC: 0.7522 - binary_accuracy: 0.0789 - val_loss: 2.4430 - val_average_precision: 0.1351 - val_AUCROC: 0.7011 - val_binary_accuracy: 0.0656
Epoch 4/200
7/7 [==============================] - 0s 27ms/step - loss: 2.3120 - average_precision: 0.1965 - AUCROC: 0.7566 - binary_accuracy: 0.0789 - val_loss: 2.2344 - val_average_precision: 0.1377 - val_AUCROC: 0.7078 - val_binary_accuracy:

7/7 [==============================] - 0s 43ms/step - loss: 0.2496 - average_precision: 0.1928 - AUCROC: 0.7767 - binary_accuracy: 0.9211 - val_loss: 0.2263 - val_average_precision: 0.1361 - val_AUCROC: 0.7224 - val_binary_accuracy: 0.9344
Epoch 34/200
7/7 [==============================] - 0s 29ms/step - loss: 0.2482 - average_precision: 0.2004 - AUCROC: 0.7793 - binary_accuracy: 0.9211 - val_loss: 0.2267 - val_average_precision: 0.1371 - val_AUCROC: 0.7263 - val_binary_accuracy: 0.9344
Epoch 35/200
7/7 [==============================] - 0s 30ms/step - loss: 0.2470 - average_precision: 0.2205 - AUCROC: 0.7796 - binary_accuracy: 0.9211 - val_loss: 0.2252 - val_average_precision: 0.1370 - val_AUCROC: 0.7294 - val_binary_accuracy: 0.9344
Epoch 36/200
7/7 [==============================] - 0s 29ms/step - loss: 0.2458 - average_precision: 0.2096 - AUCROC: 0.7798 - binary_accuracy: 0.9211 - val_loss: 0.2254 - val_average_precision: 0.1435 - val_AUCROC: 0.7314 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 29ms/step - loss: 0.2363 - average_precision: 0.2084 - AUCROC: 0.7876 - binary_accuracy: 0.9211 - val_loss: 0.2218 - val_average_precision: 0.1437 - val_AUCROC: 0.7393 - val_binary_accuracy: 0.9344
Epoch 66/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2363 - average_precision: 0.2175 - AUCROC: 0.7866 - binary_accuracy: 0.9211 - val_loss: 0.2215 - val_average_precision: 0.1437 - val_AUCROC: 0.7377 - val_binary_accuracy: 0.9344
Epoch 67/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2365 - average_precision: 0.2184 - AUCROC: 0.7865 - binary_accuracy: 0.9211 - val_loss: 0.2223 - val_average_precision: 0.1437 - val_AUCROC: 0.7394 - val_binary_accuracy: 0.9344
Epoch 68/200
7/7 [==============================] - 0s 29ms/step - loss: 0.2364 - average_precision: 0.2101 - AUCROC: 0.7868 - binary_accuracy: 0.9211 - val_loss: 0.2212 - val_average_precision: 0.1438 - val_AUCROC: 0.7397 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 32ms/step - loss: 0.2358 - average_precision: 0.2095 - AUCROC: 0.7893 - binary_accuracy: 0.9211 - val_loss: 0.2206 - val_average_precision: 0.1451 - val_AUCROC: 0.7449 - val_binary_accuracy: 0.9344
Epoch 98/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2358 - average_precision: 0.2136 - AUCROC: 0.7894 - binary_accuracy: 0.9211 - val_loss: 0.2218 - val_average_precision: 0.1455 - val_AUCROC: 0.7451 - val_binary_accuracy: 0.9344
Epoch 99/200
7/7 [==============================] - 0s 29ms/step - loss: 0.2356 - average_precision: 0.2159 - AUCROC: 0.7895 - binary_accuracy: 0.9211 - val_loss: 0.2213 - val_average_precision: 0.1455 - val_AUCROC: 0.7461 - val_binary_accuracy: 0.9344
Epoch 100/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2356 - average_precision: 0.2229 - AUCROC: 0.7897 - binary_accuracy: 0.9211 - val_loss: 0.2260 - val_average_precision: 0.1454 - val_AUCROC: 0.7456 - val_binary_accuracy: 0.9344


7/7 [==============================] - 0s 27ms/step - loss: 0.2355 - average_precision: 0.2252 - AUCROC: 0.7897 - binary_accuracy: 0.9211 - val_loss: 0.2200 - val_average_precision: 0.1463 - val_AUCROC: 0.7469 - val_binary_accuracy: 0.9344
Epoch 130/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2355 - average_precision: 0.2118 - AUCROC: 0.7899 - binary_accuracy: 0.9211 - val_loss: 0.2213 - val_average_precision: 0.1463 - val_AUCROC: 0.7469 - val_binary_accuracy: 0.9344
Epoch 131/200
7/7 [==============================] - 0s 29ms/step - loss: 0.2354 - average_precision: 0.2121 - AUCROC: 0.7914 - binary_accuracy: 0.9211 - val_loss: 0.2193 - val_average_precision: 0.1462 - val_AUCROC: 0.7460 - val_binary_accuracy: 0.9344
Epoch 132/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2358 - average_precision: 0.2164 - AUCROC: 0.7895 - binary_accuracy: 0.9211 - val_loss: 0.2197 - val_average_precision: 0.1462 - val_AUCROC: 0.7444 - val_binary_accuracy: 0.934

...

7/7 [==============================] - 0s 31ms/step - loss: 0.2314 - average_precision: 0.2037 - AUCROC: 0.7772 - binary_accuracy: 0.9248 - val_loss: 0.2424 - val_average_precision: 0.2212 - val_AUCROC: 0.7783 - val_binary_accuracy: 0.9194
Epoch 65/200
7/7 [==============================] - 0s 34ms/step - loss: 0.2311 - average_precision: 0.1962 - AUCROC: 0.7772 - binary_accuracy: 0.9248 - val_loss: 0.2425 - val_average_precision: 0.2212 - val_AUCROC: 0.7759 - val_binary_accuracy: 0.9194
Epoch 66/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2310 - average_precision: 0.1973 - AUCROC: 0.7784 - binary_accuracy: 0.9248 - val_loss: 0.2424 - val_average_precision: 0.2212 - val_AUCROC: 0.7760 - val_binary_accuracy: 0.9194
Epoch 67/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2312 - average_precision: 0.1906 - AUCROC: 0.7775 - binary_accuracy: 0.9248 - val_loss: 0.2423 - val_average_precision: 0.2212 - val_AUCROC: 0.7781 - val_binary_accuracy: 0.9194
E

7/7 [==============================] - 0s 25ms/step - loss: 0.2303 - average_precision: 0.1942 - AUCROC: 0.7811 - binary_accuracy: 0.9248 - val_loss: 0.2421 - val_average_precision: 0.2209 - val_AUCROC: 0.7775 - val_binary_accuracy: 0.9194
Epoch 97/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2302 - average_precision: 0.1909 - AUCROC: 0.7822 - binary_accuracy: 0.9248 - val_loss: 0.2434 - val_average_precision: 0.2209 - val_AUCROC: 0.7773 - val_binary_accuracy: 0.9194
Epoch 98/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2307 - average_precision: 0.2175 - AUCROC: 0.7792 - binary_accuracy: 0.9248 - val_loss: 0.2421 - val_average_precision: 0.2209 - val_AUCROC: 0.7757 - val_binary_accuracy: 0.9194
Epoch 99/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2302 - average_precision: 0.2059 - AUCROC: 0.7824 - binary_accuracy: 0.9248 - val_loss: 0.2431 - val_average_precision: 0.2209 - val_AUCROC: 0.7780 - val_binary_accuracy: 0.9194
E

7/7 [==============================] - 0s 23ms/step - loss: 0.2297 - average_precision: 0.1920 - AUCROC: 0.7829 - binary_accuracy: 0.9248 - val_loss: 0.2459 - val_average_precision: 0.2202 - val_AUCROC: 0.7768 - val_binary_accuracy: 0.9194
Epoch 129/200
7/7 [==============================] - 0s 29ms/step - loss: 0.2304 - average_precision: 0.2057 - AUCROC: 0.7826 - binary_accuracy: 0.9248 - val_loss: 0.2421 - val_average_precision: 0.2201 - val_AUCROC: 0.7756 - val_binary_accuracy: 0.9194
Epoch 130/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2300 - average_precision: 0.2007 - AUCROC: 0.7812 - binary_accuracy: 0.9248 - val_loss: 0.2423 - val_average_precision: 0.2201 - val_AUCROC: 0.7770 - val_binary_accuracy: 0.9194
Epoch 131/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2304 - average_precision: 0.1999 - AUCROC: 0.7802 - binary_accuracy: 0.9248 - val_loss: 0.2420 - val_average_precision: 0.2202 - val_AUCROC: 0.7757 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 27ms/step - loss: 0.2297 - average_precision: 0.1912 - AUCROC: 0.7819 - binary_accuracy: 0.9248 - val_loss: 0.2430 - val_average_precision: 0.2195 - val_AUCROC: 0.7778 - val_binary_accuracy: 0.9194
Epoch 161/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2297 - average_precision: 0.2111 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2422 - val_average_precision: 0.2206 - val_AUCROC: 0.7751 - val_binary_accuracy: 0.9194
Epoch 162/200
7/7 [==============================] - 0s 33ms/step - loss: 0.2298 - average_precision: 0.2021 - AUCROC: 0.7823 - binary_accuracy: 0.9248 - val_loss: 0.2422 - val_average_precision: 0.2206 - val_AUCROC: 0.7750 - val_binary_accuracy: 0.9194
Epoch 163/200
7/7 [==============================] - 0s 34ms/step - loss: 0.2296 - average_precision: 0.1965 - AUCROC: 0.7827 - binary_accuracy: 0.9248 - val_loss: 0.2422 - val_average_precision: 0.2206 - val_AUCROC: 0.7744 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 35ms/step - loss: 0.2297 - average_precision: 0.2026 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2427 - val_average_precision: 0.2205 - val_AUCROC: 0.7767 - val_binary_accuracy: 0.9194
Epoch 193/200
7/7 [==============================] - 0s 33ms/step - loss: 0.2298 - average_precision: 0.2007 - AUCROC: 0.7817 - binary_accuracy: 0.9248 - val_loss: 0.2425 - val_average_precision: 0.2205 - val_AUCROC: 0.7754 - val_binary_accuracy: 0.9194
Epoch 194/200
7/7 [==============================] - 0s 31ms/step - loss: 0.2295 - average_precision: 0.2005 - AUCROC: 0.7826 - binary_accuracy: 0.9248 - val_loss: 0.2423 - val_average_precision: 0.2204 - val_AUCROC: 0.7764 - val_binary_accuracy: 0.9194
Epoch 195/200
7/7 [==============================] - 0s 29ms/step - loss: 0.2295 - average_precision: 0.1975 - AUCROC: 0.7831 - binary_accuracy: 0.9248 - val_loss: 0.2425 - val_average_precision: 0.2205 - val_AUCROC: 0.7761 - val_binary_accuracy: 0.919

### Downsample Case

In [30]:
avg_precision_cross_val_down, roc_auc_cross_val_down, accuracy_cross_val_down = obtain_nn_scores(df_train, resampling = "Down")
print()
info(avg_precision_cross_val_down, roc_auc_cross_val_down, accuracy_cross_val_down)

Epoch 1/200
7/7 [==============================] - 0s 41ms/step - loss: 0.3122 - average_precision: 0.0544 - AUCROC: 0.3193 - binary_accuracy: 0.9211 - val_loss: 0.2701 - val_average_precision: 0.0464 - val_AUCROC: 0.3307 - val_binary_accuracy: 0.9344
Epoch 2/200
7/7 [==============================] - 0s 49ms/step - loss: 0.2967 - average_precision: 0.0581 - AUCROC: 0.3348 - binary_accuracy: 0.9211 - val_loss: 0.2606 - val_average_precision: 0.0483 - val_AUCROC: 0.3591 - val_binary_accuracy: 0.9344
Epoch 3/200
7/7 [==============================] - 0s 33ms/step - loss: 0.2898 - average_precision: 0.0609 - AUCROC: 0.3692 - binary_accuracy: 0.9211 - val_loss: 0.2547 - val_average_precision: 0.0490 - val_AUCROC: 0.3777 - val_binary_accuracy: 0.9344
Epoch 4/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2851 - average_precision: 0.0635 - AUCROC: 0.3990 - binary_accuracy: 0.9211 - val_loss: 0.2513 - val_average_precision: 0.0534 - val_AUCROC: 0.4106 - val_binary_accuracy:

7/7 [==============================] - 0s 27ms/step - loss: 0.2441 - average_precision: 0.1827 - AUCROC: 0.7589 - binary_accuracy: 0.9211 - val_loss: 0.2256 - val_average_precision: 0.1309 - val_AUCROC: 0.7086 - val_binary_accuracy: 0.9344
Epoch 34/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2439 - average_precision: 0.2009 - AUCROC: 0.7607 - binary_accuracy: 0.9211 - val_loss: 0.2260 - val_average_precision: 0.1314 - val_AUCROC: 0.7077 - val_binary_accuracy: 0.9344
Epoch 35/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2434 - average_precision: 0.2049 - AUCROC: 0.7616 - binary_accuracy: 0.9211 - val_loss: 0.2268 - val_average_precision: 0.1313 - val_AUCROC: 0.7060 - val_binary_accuracy: 0.9344
Epoch 36/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2431 - average_precision: 0.1887 - AUCROC: 0.7619 - binary_accuracy: 0.9211 - val_loss: 0.2253 - val_average_precision: 0.1318 - val_AUCROC: 0.7084 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 27ms/step - loss: 0.2380 - average_precision: 0.2297 - AUCROC: 0.7777 - binary_accuracy: 0.9211 - val_loss: 0.2231 - val_average_precision: 0.1381 - val_AUCROC: 0.7285 - val_binary_accuracy: 0.9344
Epoch 66/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2378 - average_precision: 0.2067 - AUCROC: 0.7767 - binary_accuracy: 0.9211 - val_loss: 0.2231 - val_average_precision: 0.1409 - val_AUCROC: 0.7292 - val_binary_accuracy: 0.9344
Epoch 67/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2378 - average_precision: 0.2026 - AUCROC: 0.7794 - binary_accuracy: 0.9211 - val_loss: 0.2238 - val_average_precision: 0.1414 - val_AUCROC: 0.7299 - val_binary_accuracy: 0.9344
Epoch 68/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2377 - average_precision: 0.1985 - AUCROC: 0.7789 - binary_accuracy: 0.9211 - val_loss: 0.2229 - val_average_precision: 0.1423 - val_AUCROC: 0.7316 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 25ms/step - loss: 0.2359 - average_precision: 0.2192 - AUCROC: 0.7879 - binary_accuracy: 0.9211 - val_loss: 0.2234 - val_average_precision: 0.1416 - val_AUCROC: 0.7395 - val_binary_accuracy: 0.9344
Epoch 98/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2358 - average_precision: 0.2061 - AUCROC: 0.7891 - binary_accuracy: 0.9211 - val_loss: 0.2215 - val_average_precision: 0.1434 - val_AUCROC: 0.7430 - val_binary_accuracy: 0.9344
Epoch 99/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2358 - average_precision: 0.2258 - AUCROC: 0.7880 - binary_accuracy: 0.9211 - val_loss: 0.2232 - val_average_precision: 0.1432 - val_AUCROC: 0.7418 - val_binary_accuracy: 0.9344
Epoch 100/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2358 - average_precision: 0.2066 - AUCROC: 0.7887 - binary_accuracy: 0.9211 - val_loss: 0.2221 - val_average_precision: 0.1439 - val_AUCROC: 0.7387 - val_binary_accuracy: 0.9344


7/7 [==============================] - 0s 27ms/step - loss: 0.2353 - average_precision: 0.2206 - AUCROC: 0.7906 - binary_accuracy: 0.9211 - val_loss: 0.2214 - val_average_precision: 0.1439 - val_AUCROC: 0.7440 - val_binary_accuracy: 0.9344
Epoch 130/200
7/7 [==============================] - 0s 33ms/step - loss: 0.2354 - average_precision: 0.2130 - AUCROC: 0.7893 - binary_accuracy: 0.9211 - val_loss: 0.2214 - val_average_precision: 0.1440 - val_AUCROC: 0.7441 - val_binary_accuracy: 0.9344
Epoch 131/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2354 - average_precision: 0.2221 - AUCROC: 0.7903 - binary_accuracy: 0.9211 - val_loss: 0.2211 - val_average_precision: 0.1437 - val_AUCROC: 0.7449 - val_binary_accuracy: 0.9344
Epoch 132/200
7/7 [==============================] - 0s 30ms/step - loss: 0.2354 - average_precision: 0.2221 - AUCROC: 0.7893 - binary_accuracy: 0.9211 - val_loss: 0.2232 - val_average_precision: 0.1439 - val_AUCROC: 0.7454 - val_binary_accuracy: 0.934

...

7/7 [==============================] - 0s 23ms/step - loss: 0.2303 - average_precision: 0.2042 - AUCROC: 0.7810 - binary_accuracy: 0.9248 - val_loss: 0.2437 - val_average_precision: 0.2399 - val_AUCROC: 0.7768 - val_binary_accuracy: 0.9194
Epoch 66/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2304 - average_precision: 0.2021 - AUCROC: 0.7812 - binary_accuracy: 0.9248 - val_loss: 0.2431 - val_average_precision: 0.2401 - val_AUCROC: 0.7760 - val_binary_accuracy: 0.9194
Epoch 67/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2303 - average_precision: 0.2016 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2436 - val_average_precision: 0.2400 - val_AUCROC: 0.7748 - val_binary_accuracy: 0.9194
Epoch 68/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2303 - average_precision: 0.1990 - AUCROC: 0.7812 - binary_accuracy: 0.9248 - val_loss: 0.2432 - val_average_precision: 0.2400 - val_AUCROC: 0.7779 - val_binary_accuracy: 0.9194
E

7/7 [==============================] - 0s 24ms/step - loss: 0.2299 - average_precision: 0.1999 - AUCROC: 0.7828 - binary_accuracy: 0.9248 - val_loss: 0.2430 - val_average_precision: 0.2394 - val_AUCROC: 0.7748 - val_binary_accuracy: 0.9194
Epoch 98/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2300 - average_precision: 0.2010 - AUCROC: 0.7823 - binary_accuracy: 0.9248 - val_loss: 0.2438 - val_average_precision: 0.2391 - val_AUCROC: 0.7757 - val_binary_accuracy: 0.9194
Epoch 99/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2300 - average_precision: 0.1958 - AUCROC: 0.7815 - binary_accuracy: 0.9248 - val_loss: 0.2430 - val_average_precision: 0.2391 - val_AUCROC: 0.7757 - val_binary_accuracy: 0.9194
Epoch 100/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2300 - average_precision: 0.2159 - AUCROC: 0.7812 - binary_accuracy: 0.9248 - val_loss: 0.2427 - val_average_precision: 0.2386 - val_AUCROC: 0.7768 - val_binary_accuracy: 0.9194


7/7 [==============================] - 0s 28ms/step - loss: 0.2301 - average_precision: 0.1985 - AUCROC: 0.7817 - binary_accuracy: 0.9248 - val_loss: 0.2426 - val_average_precision: 0.2384 - val_AUCROC: 0.7784 - val_binary_accuracy: 0.9194
Epoch 130/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2299 - average_precision: 0.2091 - AUCROC: 0.7822 - binary_accuracy: 0.9248 - val_loss: 0.2425 - val_average_precision: 0.2389 - val_AUCROC: 0.7767 - val_binary_accuracy: 0.9194
Epoch 131/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2299 - average_precision: 0.1958 - AUCROC: 0.7822 - binary_accuracy: 0.9248 - val_loss: 0.2432 - val_average_precision: 0.2389 - val_AUCROC: 0.7781 - val_binary_accuracy: 0.9194
Epoch 132/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2299 - average_precision: 0.2019 - AUCROC: 0.7826 - binary_accuracy: 0.9248 - val_loss: 0.2428 - val_average_precision: 0.2389 - val_AUCROC: 0.7745 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 23ms/step - loss: 0.2299 - average_precision: 0.2013 - AUCROC: 0.7826 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2396 - val_AUCROC: 0.7774 - val_binary_accuracy: 0.9194
Epoch 162/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2301 - average_precision: 0.1950 - AUCROC: 0.7807 - binary_accuracy: 0.9248 - val_loss: 0.2427 - val_average_precision: 0.2395 - val_AUCROC: 0.7764 - val_binary_accuracy: 0.9194
Epoch 163/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2299 - average_precision: 0.1986 - AUCROC: 0.7818 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2396 - val_AUCROC: 0.7777 - val_binary_accuracy: 0.9194
Epoch 164/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2299 - average_precision: 0.1962 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2427 - val_average_precision: 0.2403 - val_AUCROC: 0.7754 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 29ms/step - loss: 0.2300 - average_precision: 0.2154 - AUCROC: 0.7824 - binary_accuracy: 0.9248 - val_loss: 0.2428 - val_average_precision: 0.2396 - val_AUCROC: 0.7767 - val_binary_accuracy: 0.9194
Epoch 194/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2298 - average_precision: 0.2011 - AUCROC: 0.7827 - binary_accuracy: 0.9248 - val_loss: 0.2425 - val_average_precision: 0.2395 - val_AUCROC: 0.7765 - val_binary_accuracy: 0.9194
Epoch 195/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2298 - average_precision: 0.2212 - AUCROC: 0.7826 - binary_accuracy: 0.9248 - val_loss: 0.2425 - val_average_precision: 0.2387 - val_AUCROC: 0.7777 - val_binary_accuracy: 0.9194
Epoch 196/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2298 - average_precision: 0.2033 - AUCROC: 0.7829 - binary_accuracy: 0.9248 - val_loss: 0.2427 - val_average_precision: 0.2395 - val_AUCROC: 0.7770 - val_binary_accuracy: 0.919

### Balanced Sampling Case

In [31]:
avg_precision_cross_val_bal, roc_auc_cross_val_bal, accuracy_cross_val_bal = obtain_nn_scores(df_train, resampling = "Balanced")
print()
info(avg_precision_cross_val_bal, roc_auc_cross_val_bal, accuracy_cross_val_bal)

Epoch 1/200
7/7 [==============================] - 0s 34ms/step - loss: 2.7983 - average_precision: 0.1292 - AUCROC: 0.6724 - binary_accuracy: 0.0789 - val_loss: 2.6607 - val_average_precision: 0.0997 - val_AUCROC: 0.6298 - val_binary_accuracy: 0.0656
Epoch 2/200
7/7 [==============================] - 0s 26ms/step - loss: 2.5415 - average_precision: 0.1414 - AUCROC: 0.6800 - binary_accuracy: 0.0789 - val_loss: 2.4607 - val_average_precision: 0.0997 - val_AUCROC: 0.6318 - val_binary_accuracy: 0.0656
Epoch 3/200
7/7 [==============================] - 0s 23ms/step - loss: 2.3584 - average_precision: 0.1417 - AUCROC: 0.6711 - binary_accuracy: 0.0789 - val_loss: 2.2920 - val_average_precision: 0.0996 - val_AUCROC: 0.6146 - val_binary_accuracy: 0.0656
Epoch 4/200
7/7 [==============================] - 0s 32ms/step - loss: 2.1984 - average_precision: 0.1280 - AUCROC: 0.6734 - binary_accuracy: 0.0789 - val_loss: 2.1372 - val_average_precision: 0.0997 - val_AUCROC: 0.6259 - val_binary_accuracy:

7/7 [==============================] - 0s 22ms/step - loss: 0.2528 - average_precision: 0.2000 - AUCROC: 0.7697 - binary_accuracy: 0.9211 - val_loss: 0.2391 - val_average_precision: 0.1336 - val_AUCROC: 0.7145 - val_binary_accuracy: 0.9344
Epoch 34/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2470 - average_precision: 0.2117 - AUCROC: 0.7715 - binary_accuracy: 0.9211 - val_loss: 0.2340 - val_average_precision: 0.1338 - val_AUCROC: 0.7165 - val_binary_accuracy: 0.9344
Epoch 35/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2436 - average_precision: 0.1956 - AUCROC: 0.7701 - binary_accuracy: 0.9211 - val_loss: 0.2302 - val_average_precision: 0.1331 - val_AUCROC: 0.7182 - val_binary_accuracy: 0.9344
Epoch 36/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2412 - average_precision: 0.1974 - AUCROC: 0.7714 - binary_accuracy: 0.9211 - val_loss: 0.2275 - val_average_precision: 0.1336 - val_AUCROC: 0.7176 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 24ms/step - loss: 0.2368 - average_precision: 0.1973 - AUCROC: 0.7758 - binary_accuracy: 0.9211 - val_loss: 0.2250 - val_average_precision: 0.1371 - val_AUCROC: 0.7243 - val_binary_accuracy: 0.9344
Epoch 66/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2369 - average_precision: 0.1970 - AUCROC: 0.7746 - binary_accuracy: 0.9211 - val_loss: 0.2244 - val_average_precision: 0.1371 - val_AUCROC: 0.7246 - val_binary_accuracy: 0.9344
Epoch 67/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2367 - average_precision: 0.2028 - AUCROC: 0.7754 - binary_accuracy: 0.9211 - val_loss: 0.2252 - val_average_precision: 0.1360 - val_AUCROC: 0.7287 - val_binary_accuracy: 0.9344
Epoch 68/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2367 - average_precision: 0.2118 - AUCROC: 0.7756 - binary_accuracy: 0.9211 - val_loss: 0.2250 - val_average_precision: 0.1371 - val_AUCROC: 0.7284 - val_binary_accuracy: 0.9344
E

7/7 [==============================] - 0s 26ms/step - loss: 0.2359 - average_precision: 0.2235 - AUCROC: 0.7837 - binary_accuracy: 0.9211 - val_loss: 0.2230 - val_average_precision: 0.1444 - val_AUCROC: 0.7397 - val_binary_accuracy: 0.9344
Epoch 98/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2358 - average_precision: 0.2097 - AUCROC: 0.7844 - binary_accuracy: 0.9211 - val_loss: 0.2227 - val_average_precision: 0.1456 - val_AUCROC: 0.7374 - val_binary_accuracy: 0.9344
Epoch 99/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2359 - average_precision: 0.2094 - AUCROC: 0.7848 - binary_accuracy: 0.9211 - val_loss: 0.2231 - val_average_precision: 0.1455 - val_AUCROC: 0.7450 - val_binary_accuracy: 0.9344
Epoch 100/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2358 - average_precision: 0.2151 - AUCROC: 0.7854 - binary_accuracy: 0.9211 - val_loss: 0.2228 - val_average_precision: 0.1450 - val_AUCROC: 0.7418 - val_binary_accuracy: 0.9344


7/7 [==============================] - 0s 25ms/step - loss: 0.2356 - average_precision: 0.2016 - AUCROC: 0.7874 - binary_accuracy: 0.9211 - val_loss: 0.2225 - val_average_precision: 0.1438 - val_AUCROC: 0.7434 - val_binary_accuracy: 0.9344
Epoch 130/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2355 - average_precision: 0.2145 - AUCROC: 0.7890 - binary_accuracy: 0.9211 - val_loss: 0.2226 - val_average_precision: 0.1439 - val_AUCROC: 0.7433 - val_binary_accuracy: 0.9344
Epoch 131/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2355 - average_precision: 0.2188 - AUCROC: 0.7886 - binary_accuracy: 0.9211 - val_loss: 0.2225 - val_average_precision: 0.1438 - val_AUCROC: 0.7461 - val_binary_accuracy: 0.9344
Epoch 132/200
7/7 [==============================] - 0s 28ms/step - loss: 0.2355 - average_precision: 0.2120 - AUCROC: 0.7887 - binary_accuracy: 0.9211 - val_loss: 0.2226 - val_average_precision: 0.1438 - val_AUCROC: 0.7442 - val_binary_accuracy: 0.934

...

7/7 [==============================] - 0s 27ms/step - loss: 0.2301 - average_precision: 0.1938 - AUCROC: 0.7809 - binary_accuracy: 0.9248 - val_loss: 0.2431 - val_average_precision: 0.2297 - val_AUCROC: 0.7760 - val_binary_accuracy: 0.9194
Epoch 66/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2301 - average_precision: 0.1969 - AUCROC: 0.7827 - binary_accuracy: 0.9248 - val_loss: 0.2428 - val_average_precision: 0.2062 - val_AUCROC: 0.7701 - val_binary_accuracy: 0.9194
Epoch 67/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2303 - average_precision: 0.2018 - AUCROC: 0.7798 - binary_accuracy: 0.9248 - val_loss: 0.2430 - val_average_precision: 0.2297 - val_AUCROC: 0.7758 - val_binary_accuracy: 0.9194
Epoch 68/200
7/7 [==============================] - 0s 26ms/step - loss: 0.2301 - average_precision: 0.2014 - AUCROC: 0.7815 - binary_accuracy: 0.9248 - val_loss: 0.2427 - val_average_precision: 0.2296 - val_AUCROC: 0.7735 - val_binary_accuracy: 0.9194
E

7/7 [==============================] - 0s 24ms/step - loss: 0.2302 - average_precision: 0.2018 - AUCROC: 0.7804 - binary_accuracy: 0.9248 - val_loss: 0.2433 - val_average_precision: 0.2294 - val_AUCROC: 0.7762 - val_binary_accuracy: 0.9194
Epoch 98/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2299 - average_precision: 0.2121 - AUCROC: 0.7819 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2296 - val_AUCROC: 0.7763 - val_binary_accuracy: 0.9194
Epoch 99/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2297 - average_precision: 0.1970 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2428 - val_average_precision: 0.2296 - val_AUCROC: 0.7734 - val_binary_accuracy: 0.9194
Epoch 100/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2299 - average_precision: 0.2006 - AUCROC: 0.7813 - binary_accuracy: 0.9248 - val_loss: 0.2431 - val_average_precision: 0.2290 - val_AUCROC: 0.7768 - val_binary_accuracy: 0.9194


7/7 [==============================] - 0s 27ms/step - loss: 0.2297 - average_precision: 0.2007 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2430 - val_average_precision: 0.2296 - val_AUCROC: 0.7766 - val_binary_accuracy: 0.9194
Epoch 130/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2298 - average_precision: 0.1876 - AUCROC: 0.7812 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2293 - val_AUCROC: 0.7762 - val_binary_accuracy: 0.9194
Epoch 131/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2297 - average_precision: 0.1975 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2296 - val_AUCROC: 0.7768 - val_binary_accuracy: 0.9194
Epoch 132/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2297 - average_precision: 0.2064 - AUCROC: 0.7831 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2299 - val_AUCROC: 0.7727 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 25ms/step - loss: 0.2297 - average_precision: 0.1922 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2296 - val_AUCROC: 0.7772 - val_binary_accuracy: 0.9194
Epoch 162/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2298 - average_precision: 0.2012 - AUCROC: 0.7813 - binary_accuracy: 0.9248 - val_loss: 0.2432 - val_average_precision: 0.2297 - val_AUCROC: 0.7753 - val_binary_accuracy: 0.9194
Epoch 163/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2298 - average_precision: 0.1986 - AUCROC: 0.7816 - binary_accuracy: 0.9248 - val_loss: 0.2432 - val_average_precision: 0.2296 - val_AUCROC: 0.7756 - val_binary_accuracy: 0.9194
Epoch 164/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2296 - average_precision: 0.1879 - AUCROC: 0.7827 - binary_accuracy: 0.9248 - val_loss: 0.2430 - val_average_precision: 0.2298 - val_AUCROC: 0.7769 - val_binary_accuracy: 0.919

7/7 [==============================] - 0s 25ms/step - loss: 0.2298 - average_precision: 0.1983 - AUCROC: 0.7821 - binary_accuracy: 0.9248 - val_loss: 0.2431 - val_average_precision: 0.2399 - val_AUCROC: 0.7759 - val_binary_accuracy: 0.9194
Epoch 194/200
7/7 [==============================] - 0s 25ms/step - loss: 0.2298 - average_precision: 0.2052 - AUCROC: 0.7811 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2393 - val_AUCROC: 0.7769 - val_binary_accuracy: 0.9194
Epoch 195/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2299 - average_precision: 0.2071 - AUCROC: 0.7812 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2400 - val_AUCROC: 0.7767 - val_binary_accuracy: 0.9194
Epoch 196/200
7/7 [==============================] - 0s 27ms/step - loss: 0.2297 - average_precision: 0.2259 - AUCROC: 0.7818 - binary_accuracy: 0.9248 - val_loss: 0.2429 - val_average_precision: 0.2393 - val_AUCROC: 0.7763 - val_binary_accuracy: 0.919